In [1]:
import warnings
# Test importing PandasSeries
#Run me
from snowflake.ml.modeling.xgboost import XGBClassifier
from snowflake.ml.utils.connection_params import SnowflakeLoginOptions
from snowflake.snowpark import Session
from snowflake.snowpark import types as T
from snowflake.snowpark.functions import col
from snowflake.ml.modeling.impute import SimpleImputer
from snowflake.ml.modeling.metrics import accuracy_score, precision_score, recall_score
from snowflake.ml.modeling.preprocessing import OneHotEncoder

warnings.simplefilter(action="ignore", category=UserWarning)

In [2]:
session = Session.builder.configs(SnowflakeLoginOptions()).getOrCreate()

SnowflakeLoginOptions() is in private preview since 0.2.0. Do not use it in production. 


Error: Default connection with name 'default' cannot be found, known ones are []

In [3]:
titanic_df = session.table("titanic")

NameError: name 'session' is not defined

In [ ]:
titanic_df.show()

In [ ]:
# Columns with null values and their respective counts
{
    k: v
    for k, v in {
        col_name: titanic_df.where(col(col_name).is_null()).count()
        for col_name in titanic_df.columns
    }.items()
    if v > 0
}

In [ ]:
titanic_df = titanic_df.drop(["AGE", "DECK", "ALIVE", "ADULT_MALE", "EMBARKED"])

In [ ]:
titanic_df = titanic_df.withColumn("FARE", titanic_df["FARE"].astype(T.FloatType()))

titanic_df.show()

In [ ]:
cat_cols = ["SEX", "CLASS", "WHO", "EMBARK_TOWN"]
num_cols = ["PCLASS", "SIBSP", "PARCH", "FARE"]

In [ ]:
impute_cat = SimpleImputer(
    input_cols=cat_cols,
    output_cols=cat_cols,
    strategy="most_frequent",
    drop_input_cols=True,
)

titanic_df = impute_cat.fit(titanic_df).transform(titanic_df)
titanic_df.show()

In [ ]:
OHE = OneHotEncoder(
    input_cols=cat_cols,
    output_cols=cat_cols,
    drop_input_cols=True,
    drop="first",
    handle_unknown="ignore",
)

titanic_df = OHE.fit(titanic_df).transform(titanic_df)
titanic_df.show()

In [ ]:
train_df, test_df = titanic_df.random_split(weights=[0.8, 0.2], seed=8)

In [ ]:
xgb = XGBClassifier(
    input_cols=train_df.drop("SURVIVED").columns,
    label_cols="SURVIVED",
    output_cols="PRED_SURVIVED",
)

# Train
xgb.fit(train_df)

In [ ]:
result = xgb.predict(test_df)

In [ ]:
accuracy = accuracy_score(
    df=result, y_true_col_names="SURVIVED", y_pred_col_names="PRED_SURVIVED"
)

precision = precision_score(
    df=result, y_true_col_names="SURVIVED", y_pred_col_names="PRED_SURVIVED"
)

recall = recall_score(
    df=result, y_true_col_names="SURVIVED", y_pred_col_names="PRED_SURVIVED"
)

print(f"Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}")